# Loading data
- Initially, I had multiple datasets containing lists of only ad servers and non-ad servers. I combined them all to create a dataset 'all.csv'.
- Since all.csv had multiple overlapping entries, I deleted the duplicates and saved it as another file 'all-without-duplicates'
```
df = pd.read_csv("../lists/all.csv",converters={'domain': convert_dtype,'class': convert_dtype}) 
df = df.drop_duplicates()
df.to_csv('../lists/all-without-duplicates.csv')

```


In [7]:
import pandas as pd
#Convert dtypes for fixing Dtypewarning
# https://www.roelpeters.be/solved-dtypewarning-columns-have-mixed-types-specify-dtype-option-on-import-or-set-low-memory-in-pandas/
def convert_dtype(x):
    if not x:
        return ''
    try:
        return str(x)   
    except:        
        return ''

df = pd.read_csv("../lists/all-without-duplicates.csv",converters={'domain': convert_dtype,'class': convert_dtype}) # Dataset is now stored in a Pandas Dataframe
#df = df.drop_duplicates()
#df.to_csv('../lists/all-without-duplicates.csv')
df